## DS4M Project Code

# **Exploring Visitor Trends in Portugal: A Data-Driven Perspective**
**Data Science for Marketing** |
Outomn Semester - 2024/2025



Student Number  | Student Name
-------------------|------------------
20240108| Catarina Sousa
20240125|Maria Rita Correia
20211619       | Mariana Takimura
20241482 | Tomás Gomes

# *EuropeTop100Attractions_ENG_20190101_20210821* dataset description

## Sheet **Reviews**


| **Feature**           | **Type**    | **Description**                                                                                                              |
|------------------------|-------------|------------------------------------------------------------------------------------------------------------------------------|
| **localID**           | string      | ID of the attraction                                                                                                         |
| **extractionDate**    | date        | Date when the review was extracted                                                                                           |
| **globalRating**      | numeric     | Global rating of the attraction at the time of the review extraction (1 to 5 stars)                                          |
| **positionOnRanking** | numeric     | Position in TripAdvisor's regional ranking at the extraction date                                                            |
| **sitesOnRanking**    | numeric     | Total number of attractions in TripAdvisor's regional ranking at the extraction date                                         |
| **totalReviews**      | numeric     | Total reviews written for the attraction at the time of the review extraction                                                |
| **userName**          | string      | User name of the TripAdvisor user who posted the review. Format: first@second (public name@unique identifier)                |
| **userLocation**      | string      | Location of the user who posted the review (optional field; many users do not provide their location)                        |
| **userContributions** | numeric     | Number of reviews written by the user on TripAdvisor at the moment of extraction                                             |
| **tripType**          | string      | Type of trip (optional field)                                                                                                |
| **reviewWritten**     | date        | Date when the review was published                                                                                           |
| **reviewVisited**     | date        | Date when the customer visited the attraction. Day is always 1 (TripAdvisor only asks for year and month, not the exact day) |
| **reviewRating**      | numeric     | Quantitative rating assigned by the user (1 to 5 stars)                                                                      |
| **reviewLanguage**    | string      | Language the review was written in (always "en" for English)                                                                 |
| **reviewFullText**    | string      | Full text of the review (optional; not considered for grading as Text Mining is not covered in this course)                  |

## Sheet: Attractions

| Column Name | Data Type | Description                                       |
|-------------|-----------|---------------------------------------------------|
| ID          | string    | ID of the attraction                              |
| Name        | string    | Name of the attraction                            |
| Country     | string    | Name of the country or region                     |
| ISO         | string    | ISO code of the country or region                 |

## *Holidays.csv* dataset description

| Column Name          | Data Type   | Description                                                                 |
|----------------------|-------------|-----------------------------------------------------------------------------|
| countryOrRegion      | string      | Country or region full name                                                  |
| countryOrRegionCode  | string      | Country or region in ISO format                                              |
| date                 | date        | Date of the holiday                                                          |
| holidayName          | string      | Full name of the holiday                                                     |
| isPaidTimeOff        | boolean     | Indicates whether most people have paid time off on this date (only for US, GB, and India; NULL means unknown) |
| normalizeHolidayName | string      | Normalized name of the holiday                                               |


# Table of contents

1. [Preparing the Data](#preparing)  
  1.1. [Importing Necessary Libraries](#importing)

  1.2. [Importing Datasets](#datasets)

2. [Data Cleaning](#cleaning)  
  2.1. [Removing Duplicates](#duplicates)

  2.2. [Correcting Inconsistencies](#inconsistencies)  

  2.3. [Handling Missing Values](#missing)  

3. [Data Transformation](#transformation)  
  3.1. [Feature Engineering](#feature)  
   
  3.2. [Creating New Variables](#new)  

4. [Data Integration](#integration)  
  4.1. [Merging Datasets](#merge)  

  4.2. [Resolving Merge Conflicts](#merge-conflicts)  

5. [Data Scaling/Normalization](#scaling)  

6. [Final Dataset Review and Export](#final-review)  
  6.1. [Summary of Preprocessed Data](#summary-data)  
  6.2. [Saving the Cleaned Datasets](#saving-data)


<a name="preparing"></a>
# 1. Preparing the Data

<a name="importing"></a>
## 1.1. Importing Necessary Libraries

In [1]:
!pip install googletrans==4.0.0-rc1
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import ticker
from googletrans import Translator #translate normalizedHolidayName
!ls
from google.colab import files
from google.colab import drive

attractions_cleaned.xlsx  holidays_cleaned.xlsx		  reviews_attractions.xlsx  sample_data
drive			  Reviews_Attractions_Final.xlsx  reviews_cleaned.xlsx


<a name="datasets"></a>
## 1.2. Importing Datasets

Once again the datasets were imported seperatly and a copy was made for each dataset so that any changes would not interfer with the original data so that we can proceed our Data Preparation.

In [2]:
path_drive = "/content/drive/MyDrive/DS4M_PROJECT"
list_subfolders = os.listdir(path_drive)
list_subfolders

['Holidays.csv',
 'EuropeTop100Attractions_ENG.xlsx',
 'DS4M_GroupProject.ipynb',
 'World_Cities.xlsx',
 'Avaliações de Atrações Final.xlsx',
 'Reviews_Exported_v2.xlsx']

<font color='#FDB813'>

### **Reviews** </font> Dataset

In [3]:
ds_reviews = pd.read_excel("/content/drive/MyDrive/DS4M_PROJECT/EuropeTop100Attractions_ENG.xlsx", sheet_name="Reviews")
reviews = ds_reviews.copy()

---

<font color='#00D30F'>

### **Attractions** </font> Dataset

In [4]:
ds_attractions = pd.read_excel("/content/drive/MyDrive/DS4M_PROJECT/EuropeTop100Attractions_ENG.xlsx", sheet_name="Attractions")
attractions = ds_attractions.copy()

---

<font color='#0095DA'>

### **Holidays** </font> Dataset

In [5]:
ds_holidays = pd.read_csv("/content/drive/MyDrive/DS4M_PROJECT/Holidays.csv", sep=";",  index_col= 0)
holidays = ds_holidays.copy()

<a name="cleaning"></a>
# 2. Data Cleaning


In this section of the Notebook we will clean the data according to the insights taken from the Data Understanding Notebook.
We started by handling the duplicates and inconsistencies and finally treating the missing values. This sequence was done to possibly decrease the number values to be cleaned.

<a name="duplicates"></a>
## 2.1. Handling Duplicates


<font color='#FDB813'>

### **Reviews** </font> Dataset

In the analysis of this dataset we did not find duplicates in the dateset itself but they were found within the feature ``reviewFullText``.

To further confirm this, we created *duplicates_grouped* that contains the duplicates from ``reviewFullText`` and *displayed* it. We decided to keep the first occurence of the duplicates in the feature and *displayed* to confirm thheir removal.


In [6]:
reviews.duplicated().value_counts()

,count
False,92120


In [7]:
duplicates_grouped = reviews[reviews['reviewFullText'].duplicated(keep=False)]
display(duplicates_grouped.sort_values(by='reviewFullText'))

,localID,extractionDate,globalRating,positionOnRanking,sitesOnRanking,totalReviews,userName,userLocation,userContributions,tripType,reviewWritten,reviewVisited,reviewRating,reviewLanguage,reviewFullText
57356,MAG030,2021-08-20 17:40:35.569,4.5,2,1186,18474,GlobeTrotting828238@GlobeTrotting828238,NaN,4,NaN,2019-09-08,2019-08-01,5,en,able to attend a concert the flamenco dance...
57346,MAG030,2021-08-20 17:40:32.987,4.5,2,1186,18474,GlobeTrotting828238@GlobeTrotting828238,NaN,4,NaN,2019-09-08,2019-08-01,5,en,able to attend a concert the flamenco dance...
10036,MAG002,2021-08-20 09:59:54.757,4.0,11,1186,75909,PetDim@PetDim,"Madrid, Spain",387,Family,2019-06-09,2019-06-01,2,en,10 years ago park was free. You just walk an...
10026,MAG002,2021-08-20 09:59:52.557,4.0,11,1186,75909,PetDim@PetDim,"Madrid, Spain",387,Family,2019-06-09,2019-06-01,2,en,10 years ago park was free. You just walk an...
12138,MAG003,2021-08-20 10:18:39.220,4.5,7,2383,65646,Rdchampers@Rdchampers,"Puerto de Mazarron, Spain",101,Solo,2020-01-02,2019-03-01,5,en,Brilliant way to hear about history of Englan...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51410,MAG024,2021-08-20 16:36:24.025,4.5,1,188,23131,nuggit65@nuggit65,"London, UK",8,Couples,2019-07-10,2019-07-01,1,en,"£20 each,worth a fiver at most.packed solid,bo..."
31226,MAG011,2021-08-20 13:24:57.183,4.5,3,430,35716,dirklaren@dirklaren,"Laren, The Netherlands",305,Couples,2021-08-14,2021-08-01,5,en,…. not much more to comment or write about thi...
31236,MAG011,2021-08-20 13:25:01.171,4.5,3,430,35716,dirklaren@dirklaren,"Laren, The Netherlands",305,Couples,2021-08-14,2021-08-01,5,en,…. not much more to comment or write about thi...
41242,MAG017,2021-08-20 15:00:04.412,4.5,1,378,26475,WilliamKOnTour@WilliamKOnTour,"Cambridge, UK",3,Couples,2019-09-22,2019-09-01,5,en,"❤️❤️❤️ Stunning, lovely, amazing. What else ca..."


In [8]:
reviews = reviews.drop_duplicates(subset='reviewFullText', keep='first')
display(duplicates_grouped)

,localID,extractionDate,globalRating,positionOnRanking,sitesOnRanking,totalReviews,userName,userLocation,userContributions,tripType,reviewWritten,reviewVisited,reviewRating,reviewLanguage,reviewFullText
260,MAG001,2021-08-20 08:27:01.082,4.5,1,1186,163828,insertname@jadedbear,"Singapore, Singapore",10,Family,2020-03-09,2020-03-01,5,en,Sagrada Familia was just magical and phenomena...
261,MAG001,2021-08-20 08:27:01.098,4.5,1,1186,163828,James V@jamesv841,"Odessa, TX",4,NaN,2020-03-09,2020-03-01,5,en,"Went with Olga C, our tour was fantastic, so m..."
262,MAG001,2021-08-20 08:27:01.114,4.5,1,1186,163828,Ashleigh1505@Ashleigh1505,"Essex, UK",48,Couples,2020-03-09,2020-02-01,5,en,What a beautiful building. You honestly must s...
263,MAG001,2021-08-20 08:27:01.132,4.5,1,1186,163828,WilmaMcDermid@WilmaMcDermid,"Edinburgh, UK",41,NaN,2020-03-09,2020-02-01,4,en,We would have given it more but although it wa...
264,MAG001,2021-08-20 08:27:01.142,4.5,1,1186,163828,Jet49578@Lainey49578,"Colchester, UK",21,NaN,2020-03-09,2020-03-01,5,en,This Basilica is truly ethereal and so many be...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91993,MAG098,2021-08-21 16:25:48.936,4.5,1,58,5327,FernandoMexico8@FernandoMexico8,"Merida, Mexico",492,Couples,2019-06-09,2019-06-01,5,en,We have read all about the walls of Avila but ...
91994,MAG098,2021-08-21 16:25:49.098,4.5,1,58,5327,Globetrotter008@Globetrotter008,United Kingdom,1135,Couples,2019-06-07,2019-05-01,5,en,"The magnificent Muralla, spanning the entirety..."
91995,MAG098,2021-08-21 16:25:49.223,4.5,1,58,5327,Linda R@I1275STlindar,"Brighton, UK",9,Friends,2019-06-05,2019-06-01,5,en,The history of the wall in Avila is fascinatin...
91996,MAG098,2021-08-21 16:25:49.360,4.5,1,58,5327,Juanchoborda@Juanchoborda,"Tampa, FL",109,Couples,2019-05-31,2019-05-01,4,en,We paid the 5€ per person and got the audio gu...


---


<a name="inconsistencies"></a>
## 2.2. Correcting Inconsistencies


<font color='#FDB813'>

### **Reviews** </font> Dataset

The inconsistencies found in this dataset in the Data Understanding phase were:


``localID``:
- Strange values: "genis" and "u" were identified.

``userContributions``:
- Maximum:There is 1 entrie with the maximum that can be considered an outlier although they the 11 entries are from the same user.
- Zero value issue: There are entries with a minimum value of 0, which is illogical since users with a review in the dataset must have written at least one review. This needs to be investigated, as it could indicate missing or incorrect data.

``reviewVisited``:
- Inconsistent entries: Some reviews were written before the visit (review written date precedes the visit date). There are 22 such instances, which may be data errors or require further review.

``reviewVisited`` vs ``reviewWritten``:
- we found 22 entries where the date that the user wrote the review occured before the visitation.

Strange values were identified in ``localID`` feature, we decided to investigate them in the attractions Dataset in the feature``ID`` and confirmed that they most likely were an imputation mistake. Therefore we decided to substitute the value 'genis' with 'MAG005' and 'u' with 'MAG006'.

In [9]:
display(reviews['localID'].unique())

array(['MAG001', 'MAG002', 'MAG003', 'MAG004', 'genis', 'u', 'MAG007',
       'MAG008', 'MAG009', 'MAG010', 'MAG011', 'MAG012', 'MAG013',
       'MAG014', 'MAG015', 'MAG016', 'MAG017', 'MAG018', 'MAG019',
       'MAG020', 'MAG021', 'MAG022', 'MAG023', 'MAG024', 'MAG025',
       'MAG026', 'MAG027', 'MAG028', 'MAG029', 'MAG030', 'MAG031',
       'MAG032', 'MAG033', 'MAG034', 'MAG035', 'MAG036', 'MAG037',
       'MAG038', 'MAG039', 'MAG040', 'MAG041', 'MAG042', 'MAG043',
       'MAG044', 'MAG045', 'MAG046', 'MAG047', 'MAG048', 'MAG049',
       'MAG050', 'MAG051', 'MAG052', 'MAG053', 'MAG054', 'MAG055',
       'MAG056', 'MAG057', 'MAG058', 'MAG059', 'MAG060', 'MAG061',
       'MAG062', 'MAG063', 'MAG064', 'MAG065', 'MAG066', 'MAG067',
       'MAG068', 'MAG069', 'MAG070', 'MAG071', 'MAG072', 'MAG073',
       'MAG074', 'MAG075', 'MAG076', 'MAG077', 'MAG078', 'MAG079',
       'MAG080', 'MAG081', 'MAG082', 'MAG083', 'MAG084', 'MAG085',
       'MAG086', 'MAG087', 'MAG088', 'MAG089', 'MAG090', '

In [10]:
display(attractions['ID'].unique())

array(['MAG001', 'MAG002', 'MAG003', 'MAG004', 'MAG005', 'MAG006',
       'MAG007', 'MAG008', 'MAG009', 'MAG010', 'MAG011', 'MAG012',
       'MAG013', 'MAG014', 'MAG015', 'MAG016', 'MAG017', 'MAG018',
       'MAG019', 'MAG020', 'MAG021', 'MAG022', 'MAG023', 'MAG024',
       'MAG025', 'MAG026', 'MAG027', 'MAG028', 'MAG029', 'MAG030',
       'MAG031', 'MAG032', 'MAG033', 'MAG034', 'MAG035', 'MAG036',
       'MAG037', 'MAG038', 'MAG039', 'MAG040', 'MAG041', 'MAG042',
       'MAG043', 'MAG044', 'MAG045', 'MAG046', 'MAG047', 'MAG048',
       'MAG049', 'MAG050', 'MAG051', 'MAG052', 'MAG053', 'MAG054',
       'MAG055', 'MAG056', 'MAG057', 'MAG058', 'MAG059', 'MAG060',
       'MAG061', 'MAG062', 'MAG063', 'MAG064', 'MAG065', 'MAG066',
       'MAG067', 'MAG068', 'MAG069', 'MAG070', 'MAG071', 'MAG072',
       'MAG073', 'MAG074', 'MAG075', 'MAG076', 'MAG077', 'MAG078',
       'MAG079', 'MAG080', 'MAG081', 'MAG082', 'MAG083', 'MAG084',
       'MAG085', 'MAG086', 'MAG087', 'MAG088', 'MAG089', 'MAG0

In [11]:
reviews['localID'] = reviews['localID'].replace({'genis': 'MAG005', 'u': 'MAG006'})
display(reviews['localID'].unique())

array(['MAG001', 'MAG002', 'MAG003', 'MAG004', 'MAG005', 'MAG006',
       'MAG007', 'MAG008', 'MAG009', 'MAG010', 'MAG011', 'MAG012',
       'MAG013', 'MAG014', 'MAG015', 'MAG016', 'MAG017', 'MAG018',
       'MAG019', 'MAG020', 'MAG021', 'MAG022', 'MAG023', 'MAG024',
       'MAG025', 'MAG026', 'MAG027', 'MAG028', 'MAG029', 'MAG030',
       'MAG031', 'MAG032', 'MAG033', 'MAG034', 'MAG035', 'MAG036',
       'MAG037', 'MAG038', 'MAG039', 'MAG040', 'MAG041', 'MAG042',
       'MAG043', 'MAG044', 'MAG045', 'MAG046', 'MAG047', 'MAG048',
       'MAG049', 'MAG050', 'MAG051', 'MAG052', 'MAG053', 'MAG054',
       'MAG055', 'MAG056', 'MAG057', 'MAG058', 'MAG059', 'MAG060',
       'MAG061', 'MAG062', 'MAG063', 'MAG064', 'MAG065', 'MAG066',
       'MAG067', 'MAG068', 'MAG069', 'MAG070', 'MAG071', 'MAG072',
       'MAG073', 'MAG074', 'MAG075', 'MAG076', 'MAG077', 'MAG078',
       'MAG079', 'MAG080', 'MAG081', 'MAG082', 'MAG083', 'MAG084',
       'MAG085', 'MAG086', 'MAG087', 'MAG088', 'MAG089', 'MAG0

For the maximum value of ``userContributions`` we analyzed and discovered that it was from a single ``userName`` leading us to beleve it was an outlier and dropping it.

In [12]:
display(reviews[reviews['userContributions'] > 200000])
print((reviews[reviews['userContributions'] > 200000]).count())

,localID,extractionDate,globalRating,positionOnRanking,sitesOnRanking,totalReviews,userName,userLocation,userContributions,tripType,reviewWritten,reviewVisited,reviewRating,reviewLanguage,reviewFullText
25670,MAG007,2021-08-20 12:33:10.195,4.5,2,861,42555,Neil K@293neilk,"Liverpool, UK",607713,Friends,2019-05-29,2019-05-01,5,en,Although my twin brother Stephen and I never v...
36831,MAG013,2021-08-20 14:18:26.172,4.5,1,484,34353,Neil K@293neilk,"Liverpool, UK",607714,Friends,2019-05-18,2019-05-01,5,en,Grand Place ( Grote Markt ) is the central sq...
53994,MAG026,2021-08-20 17:05:26.785,4.5,1,309,21830,Neil K@293neilk,"Liverpool, UK",607715,Friends,2019-12-23,2019-12-01,5,en,Without a shadow of a doubt the most famous an...
56773,MAG029,2021-08-20 17:35:28.840,4.5,1,861,19017,Neil K@293neilk,"Liverpool, UK",607715,Friends,2019-05-28,2019-05-01,5,en,The Historic Center of Vienna is one of the mo...
62131,MAG035,2021-08-21 11:26:54.841,4.5,1,711,17589,Neil K@293neilk,"Liverpool, UK",607727,Friends,2021-03-20,2021-03-01,5,en,The pandemic achieved what Oliver Cromwell per...
62137,MAG035,2021-08-21 11:26:55.313,4.5,1,711,17589,Neil K@293neilk,"Liverpool, UK",607727,Friends,2020-12-13,2020-12-01,5,en,The Royal Albert Dock is without a shadow of a...
62285,MAG035,2021-08-21 11:28:21.479,4.5,1,711,17589,Neil K@293neilk,"Liverpool, UK",607727,Friends,2020-03-21,2020-03-01,5,en,The Royal Albert Dock area of Liverpool is suc...
62536,MAG035,2021-08-21 11:31:33.776,4.5,1,711,17589,Neil K@293neilk,"Liverpool, UK",607727,Friends,2019-11-26,2019-11-01,5,en,Before starting a week's holiday in Sofia my t...
75178,MAG054,2021-08-21 13:37:04.177,4.5,10,861,10250,Neil K@293neilk,"Liverpool, UK",607732,Friends,2019-05-29,2019-05-01,5,en,We had been to the wonderful Salm Brau were we...
78823,MAG065,2021-08-21 14:14:07.124,4.5,16,1110,8929,Neil K@293neilk,"Liverpool, UK",607732,Friends,2019-08-11,2019-08-01,5,en,No holiday to Berlin would be complete without...


localID              10
extractionDate       10
globalRating         10
positionOnRanking    10
sitesOnRanking       10
totalReviews         10
userName             10
userLocation         10
userContributions    10
tripType             10
reviewWritten        10
reviewVisited        10
reviewRating         10
reviewLanguage       10
reviewFullText       10
dtype: int64


In [13]:
display((reviews[reviews['userName'] == 'Neil K@293neilk'].count()))

,0
localID,10
extractionDate,10
globalRating,10
positionOnRanking,10
sitesOnRanking,10
totalReviews,10
userName,10
userLocation,10
userContributions,10
tripType,10


In [14]:
reviews = reviews[reviews['userName'] != 'Neil K@293neilk']
display(reviews[reviews['userName'] == 'Neil K@293neilk'])

,localID,extractionDate,globalRating,positionOnRanking,sitesOnRanking,totalReviews,userName,userLocation,userContributions,tripType,reviewWritten,reviewVisited,reviewRating,reviewLanguage,reviewFullText


Still treating the feature ``userContributions`` we decided that we would change the minimum values 0 to 1 since it does not make sense that a review is in the dataset and no userContributions are detected.

In [15]:
display(reviews[reviews['userContributions']==0])
print((reviews[reviews['userContributions']==0]).count())

,localID,extractionDate,globalRating,positionOnRanking,sitesOnRanking,totalReviews,userName,userLocation,userContributions,tripType,reviewWritten,reviewVisited,reviewRating,reviewLanguage,reviewFullText
18930,MAG005,2021-08-20 11:25:00.725,4.5,2,1234,55541,NaN,NaN,0,NaN,2019-03-17,NaT,5,en,No one goes to Prague without coming here. Suc...
18931,MAG005,2021-08-20 11:25:00.755,4.5,2,1234,55541,NaN,NaN,0,NaN,2019-03-16,NaT,4,en,The square is surrounded by some beautiful old...
18932,MAG005,2021-08-20 11:25:00.778,4.5,2,1234,55541,NaN,NaN,0,NaN,2019-03-16,NaT,5,en,Staroměstské Náměsti ( Old Town Square ) is t...
18933,MAG005,2021-08-20 11:25:00.802,4.5,2,1234,55541,NaN,NaN,0,NaN,2019-03-14,NaT,5,en,This is the historical center of Prague — all ...
18934,MAG005,2021-08-20 11:25:00.822,4.5,2,1234,55541,NaN,NaN,0,NaN,2019-03-13,NaT,4,en,"Having a hotel right on the Square, gave us a ..."
18935,MAG005,2021-08-20 11:25:00.843,4.5,2,1234,55541,NaN,NaN,0,NaN,2019-03-12,NaT,4,en,Very busy even in March with lots of large gro...
18936,MAG005,2021-08-20 11:25:00.863,4.5,2,1234,55541,NaN,NaN,0,NaN,2019-03-12,NaT,4,en,Our hotel was not far from this square and was...
18937,MAG005,2021-08-20 11:25:00.887,4.5,2,1234,55541,NaN,NaN,0,NaN,2019-03-11,NaT,5,en,There are so many places to visit in Prague an...
18938,MAG005,2021-08-20 11:25:00.908,4.5,2,1234,55541,NaN,NaN,0,NaN,2019-03-11,NaT,4,en,This busy old town square is a delightful coll...
18939,MAG005,2021-08-20 11:25:00.929,4.5,2,1234,55541,NaN,NaN,0,NaN,2019-03-10,NaT,1,en,Went to this stall (see photo) as the food sme...


localID              30
extractionDate       30
globalRating         30
positionOnRanking    30
sitesOnRanking       30
totalReviews         30
userName             10
userLocation          9
userContributions    30
tripType              6
reviewWritten        30
reviewVisited        10
reviewRating         30
reviewLanguage       30
reviewFullText       30
dtype: int64


In [16]:
reviews['userContributions'] = reviews['userContributions'].replace({0: 1})
print((reviews[reviews['userContributions']==0]).count())

localID              0
extractionDate       0
globalRating         0
positionOnRanking    0
sitesOnRanking       0
totalReviews         0
userName             0
userLocation         0
userContributions    0
tripType             0
reviewWritten        0
reviewVisited        0
reviewRating         0
reviewLanguage       0
reviewFullText       0
dtype: int64


In [17]:
display(((reviews[(reviews['reviewVisited'] > reviews['reviewWritten'])])).count())

,0
localID,22
extractionDate,22
globalRating,22
positionOnRanking,22
sitesOnRanking,22
totalReviews,22
userName,22
userLocation,21
userContributions,22
tripType,20


In [18]:
reviews = reviews[~(reviews['reviewVisited'] > reviews['reviewWritten'])]
display(reviews[(reviews['reviewVisited'] > reviews['reviewWritten'])])

,localID,extractionDate,globalRating,positionOnRanking,sitesOnRanking,totalReviews,userName,userLocation,userContributions,tripType,reviewWritten,reviewVisited,reviewRating,reviewLanguage,reviewFullText


---


<font color='#00D30F'>

### **Attractions** </font> Dataset

It was found inconsistencies in the features ``Country`` and ``ISO``, from the Data Understanding Notebook we extracted the 5 entries and verify their invalidity. Since they are a small number we decided to correct them mannualy.

In [19]:
display(attractions[(attractions['Country'] == 'Italy') & (attractions['ISO'] != 'IT')])
display(attractions[(attractions['Country'] == 'Poland') & (attractions['ISO'] != 'PL')])
display(attractions[(attractions['ISO'] == 'HR') & (attractions['Country'] != 'Croatia')])
display(attractions[(attractions['ISO'] == 'UK') & (attractions['Country'] != 'England')])

,ID,Name,Country,ISO
77,MAG078,Vatican City,Italy,VA


,ID,Name,Country,ISO
44,MAG045,Old Town,Poland,HR


,ID,Name,Country,ISO
44,MAG045,Old Town,Poland,HR


,ID,Name,Country,ISO
5,MAG006,Edinburgh Castle,Scotland,UK
36,MAG037,Edinburgh Old Town,Scotland,UK
62,MAG063,Calton Hill,Scot,UK


In [20]:
attractions.loc[attractions['ID'] == 'MAG078', ['ISO']] = ['IT']
attractions.loc[attractions['ID'] == 'MAG045', ['ISO']] = ['PL']
attractions.loc[attractions['ID'] == 'MAG006', ['ISO']] = ['SCT']
attractions.loc[attractions['ID'] == 'MAG037', ['ISO']] = ['SCT']
attractions.loc[attractions['ID'] == 'MAG063', ['Country']] = ['Scotland']
attractions.loc[attractions['ID'] == 'MAG063', ['ISO']] = ['SCT']
display(attractions[(attractions['Country'] == 'Italy') & (attractions['ISO'] != 'IT')])
display(attractions[(attractions['Country'] == 'Poland') & (attractions['ISO'] != 'PL')])
display(attractions[(attractions['ISO'] == 'HR') & (attractions['Country'] != 'Croatia')])
display(attractions[(attractions['ISO'] == 'UK') & (attractions['Country'] != 'England')])

,ID,Name,Country,ISO


,ID,Name,Country,ISO


,ID,Name,Country,ISO


,ID,Name,Country,ISO


In [21]:
# Remover linhas onde 'Country' é igual a 'Curaçao'
attractions = attractions[attractions['Country'] != 'Curaçao']

# Verificar o resultado
print(f"Tamanho do dataset após remoção: {attractions.shape[0]}")
display((attractions['Country'] == 'Curaçao').value_counts())


Tamanho do dataset após remoção: 99


,count
Country,
False,99


---

<font color='#0095DA'>

### **Holidays** </font> Dataset

Within this dataset the inconsistency discovered was in the feature ``date`` since our aim is to combine the three datasets, we compared holidays dataset and reviews dataset and droped the entries outside of their intersection. We also translated the ``normalizeHolidayName`` using the **translator** from **googletrans**.

In [22]:
first_holiday = holidays['date'].min()
last_holiday = holidays['date'].max()
first_review = reviews['reviewWritten'].min()
last_review = reviews['reviewWritten'].max()

print(f"Max and Min of date (holidays):\n - First holiday date: {first_holiday}\n - Last holiday date: {last_holiday}\n\nMax and Min of reviewWritten (reviews):\n - First review date: {first_review} \n - Last review date: {last_review}")

Max and Min of date (holidays):
 - First holiday date: 1970-01-01
 - Last holiday date: 2098-12-28

Max and Min of reviewWritten (reviews):
 - First review date: 2019-01-01 00:00:00 
 - Last review date: 2021-08-21 00:00:00


In [23]:
holidays['date'] = pd.to_datetime(holidays['date'])
holidays[(holidays['date'].dt.year < 2019) | (holidays['date'].dt.year > 2021)]

,countryOrRegion,holidayName,normalizeHolidayName,isPaidTimeOff,countryRegionCode,date
0,Argentina,Año Nuevo [New Year's Day],Año Nuevo [New Year's Day],NaN,AR,1970-01-01
1,Australia,New Year's Day,New Year's Day,NaN,AU,1970-01-01
2,Austria,Neujahr,Neujahr,NaN,AT,1970-01-01
3,Belgium,Nieuwjaarsdag,Nieuwjaarsdag,NaN,BE,1970-01-01
4,Brazil,Ano novo,Ano novo,NaN,BR,1970-01-01
...,...,...,...,...,...,...
69552,Sweden,Annandag jul,Annandag jul,NaN,SE,2098-12-26
69553,United Kingdom,Boxing Day,Boxing Day,True,GB,2098-12-26
69554,Wales,Boxing Day,Boxing Day,NaN,NaN,2098-12-26
69555,Norway,Søndag,Søndag,NaN,NO,2098-12-28


In [24]:
holidays = holidays[(holidays['date'] >= '2019-01-01') & (holidays['date'] <= '2021-08-21')]
display(holidays[(holidays['date'].dt.year < 2019) | (holidays['date'].dt.year > 2021)])

,countryOrRegion,holidayName,normalizeHolidayName,isPaidTimeOff,countryRegionCode,date


In [25]:
# Inicialize Translator
translator = Translator()

# Obtain unique column values
unique_names = holidays['normalizeHolidayName'].unique()
#Translate each unique name and create dictionary with translations
translated_names = {name: translator.translate(name, src='auto', dest='en').text for name in unique_names}
#Create new column with translations
holidays['translatedHolidayName'] = holidays['normalizeHolidayName'].map(translated_names)
display(holidays[['normalizeHolidayName', 'translatedHolidayName']].value_counts())

,,count
normalizeHolidayName,translatedHolidayName,
Søndag,Sunday,130
Söndag,Sunday,129
New Year's Day,New Year's Day,36
Good Friday,Good Friday,30
Boxing Day,Boxing Day,29
...,...,...
National and provincial government elections,National and provincial government elections,1
Dan sjećanja,Memorial Day,1
"Día del Veterano y de los Caidos en la Guerra de Malvinas [Veterans Day and the Fallen in the Malvinas War], Semana Santa (Viernes Santo) [Holy day (Holy Friday)]","Veteran day and the fallen in the Malvinas War [veterans day and the Fallen in the Malvinas War], Holy Week (Good Friday) [Holy Day (Holy Friday)]",1


<a name="missing"></a>
## 2.3. Handling Missing Values


<font color='#FDB813'>

### **Reviews** </font> Dataset

Has discovered in the Data Understanding Notebook, this dataset has missing values within the variables ``userName``, ``userLocation``, ``tripType`` and ``reviewVisited``:
- `userName`: 20 missing values from 92100
- `userLocation`: 13,469 missing values (~15% of the dataset) from 78651
- `tripType`: 29,068 missing values (~32% of the dataset) from 603052
- `reviewVisited`: 710 missing values (~0.8% of the dataset) from 91410

In [26]:
reviews.isnull().sum()

,0
localID,0
extractionDate,0
globalRating,0
positionOnRanking,0
sitesOnRanking,0
totalReviews,0
userName,20
userLocation,12434
userContributions,0
tripType,26910



We will use the method *.dropna()* to the variables ``userName``, and ``reviewVisited`` given that they don't make a substantial part of the dataset. Then confirm it with the *.isnull().sum()* methods combined.

In [27]:
reviews.dropna(subset=['userName', 'reviewVisited'], inplace=True)
reviews.isnull().sum()

,0
localID,0
extractionDate,0
globalRating,0
positionOnRanking,0
sitesOnRanking,0
totalReviews,0
userName,0
userLocation,12333
userContributions,0
tripType,26252


To handle the missing values for ``userLocation`` and ``tripType`` we created a function called fill_missing_values to fill the missing values from these features that had the same ``userName``.

In [28]:
# Step 1: Define a combined function to fill missing values for specified columns
def fill_missing_values(group, columns):
    for column in columns:  # Iterate over the columns to be processed
        if group[column].notnull().any():  # Check if there's at least one valid value
            valid_value = group.loc[group[column].notnull(), column].iloc[0]  # Get the first valid value
            group[column] = group[column].fillna(valid_value)  # Fill NaN values with the valid one
    return group

# Step 2: Specify the columns to treat and apply the function to each group of userName
columns_to_fill = ['userLocation', 'tripType']
reviews = reviews.groupby('userName', group_keys=False).apply(fill_missing_values, columns=columns_to_fill)

# Step 3: Verify the changes
display(reviews[reviews['userLocation'].isnull()])  # Check remaining NaN values in userLocation
display(reviews[reviews['tripType'].isnull()])      # Check remaining NaN values in tripType

<ipython-input-28-09778f2fe75b>:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  reviews = reviews.groupby('userName', group_keys=False).apply(fill_missing_values, columns=columns_to_fill)


,localID,extractionDate,globalRating,positionOnRanking,sitesOnRanking,totalReviews,userName,userLocation,userContributions,tripType,reviewWritten,reviewVisited,reviewRating,reviewLanguage,reviewFullText
3,MAG001,2021-08-20 08:24:40.147,4.5,1,1186,163828,James U@696jamesu,NaN,7,NaN,2021-08-16,2021-07-01,5,en,A MUST when visiting Barcelkona. It is consta...
6,MAG001,2021-08-20 08:24:40.190,4.5,1,1186,163828,Edwin R@edwinrL7667XX,NaN,4,NaN,2021-08-14,2021-08-01,5,en,"My 2nd trip to Barcelona, but 1st time inside ..."
9,MAG001,2021-08-20 08:24:40.272,4.5,1,1186,163828,Susan A@C9769QCsusana,NaN,4,NaN,2021-08-11,2021-08-01,5,en,I ordered up a short day trip for my daughter ...
18,MAG001,2021-08-20 08:24:46.140,4.5,1,1186,163828,Resort714982@Resort714982,NaN,3,NaN,2021-07-22,2021-07-01,5,en,"its unreal, a fairytale, you keep looking at a..."
19,MAG001,2021-08-20 08:24:46.152,4.5,1,1186,163828,Jojo in Spain@JojoinSpain,NaN,2,NaN,2021-07-21,2021-07-01,5,en,Visit at the earliest or latest ties of daylig...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92050,MAG099,2021-08-21 16:26:27.447,5.0,1,73,5345,Ines@inesbudding,NaN,3,Solo,2019-06-05,2019-05-01,5,en,I think the place Stanislas is the most beauti...
92080,MAG100,2021-08-21 16:26:53.381,4.5,11,677,5179,Hannah L@hannahlS3100LD,NaN,1,NaN,2020-02-10,2020-01-01,1,en,"Stay away, Florence is a dangerous city. I was..."
92081,MAG100,2021-08-21 16:26:53.481,4.5,11,677,5179,domodosila@donosiaul,NaN,3,Family,2019-12-28,2019-12-01,5,en,no one to manage the queue n messy ! these 2 g...
92111,MAG100,2021-08-21 16:27:11.424,4.5,11,677,5179,Ilona E@ilonae244,NaN,23,Couples,2019-04-26,2019-04-01,5,en,We fell in love with this beautiful Italian ci...


,localID,extractionDate,globalRating,positionOnRanking,sitesOnRanking,totalReviews,userName,userLocation,userContributions,tripType,reviewWritten,reviewVisited,reviewRating,reviewLanguage,reviewFullText
3,MAG001,2021-08-20 08:24:40.147,4.5,1,1186,163828,James U@696jamesu,NaN,7,NaN,2021-08-16,2021-07-01,5,en,A MUST when visiting Barcelkona. It is consta...
5,MAG001,2021-08-20 08:24:40.168,4.5,1,1186,163828,paul n@pauln3630,"Towaco, NJ",21,NaN,2021-08-15,2021-08-01,3,en,The sheer height of this church makes it a onc...
6,MAG001,2021-08-20 08:24:40.190,4.5,1,1186,163828,Edwin R@edwinrL7667XX,NaN,4,NaN,2021-08-14,2021-08-01,5,en,"My 2nd trip to Barcelona, but 1st time inside ..."
9,MAG001,2021-08-20 08:24:40.272,4.5,1,1186,163828,Susan A@C9769QCsusana,NaN,4,NaN,2021-08-11,2021-08-01,5,en,I ordered up a short day trip for my daughter ...
12,MAG001,2021-08-20 08:24:46.022,4.5,1,1186,163828,EC'sExpatLife@ECExpatLife,"Zurich, Switzerland",188,NaN,2021-08-09,2021-08-01,5,en,A must visit! The interior looks nearly compl...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92083,MAG100,2021-08-21 16:26:53.679,4.5,11,677,5179,taijitokuhisa@taijitokuhisa,"Ljubljana, Slovenia",192,NaN,2019-12-02,2019-08-01,5,en,"A city, with an incredibly rich history, where..."
92084,MAG100,2021-08-21 16:26:53.777,4.5,11,677,5179,Polonik@Polonik23,"Warsaw, Poland",6,NaN,2019-11-25,2019-11-01,5,en,"I vivited a lot of places and Florance, Firenz..."
92086,MAG100,2021-08-21 16:26:53.967,4.5,11,677,5179,tezpil@tezpil,"Portsmouth, UK",3990,NaN,2019-11-08,2019-09-01,4,en,Historically brilliant with great architecture...
92095,MAG100,2021-08-21 16:26:59.858,4.5,11,677,5179,JanuszRoman@JanuszRoman,Europe,1097,NaN,2019-09-01,2019-08-01,5,en,Centro Storico is the essence of Florence. You...


In [29]:
reviews.isnull().sum()

,0
localID,0
extractionDate,0
globalRating,0
positionOnRanking,0
sitesOnRanking,0
totalReviews,0
userName,0
userLocation,12332
userContributions,0
tripType,23373


According to the map provided by the professors, we decided that ``userLocation`` and ``tripType`` could not be removed since it is still a substancial part of the dataset or further cleaned (imputation of random values from the dataset itself) since it would skew the results, therefore we substituted the missing values with 'Unkown'.

In [30]:
reviews['userLocation'].fillna('Unkown', inplace=True)
reviews['userLocation'].value_counts(dropna=False)

<ipython-input-30-a2853c9bd384>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  reviews['userLocation'].fillna('Unkown', inplace=True)


,count
userLocation,
Unkown,12332
"London, UK",3395
"Melbourne, Australia",691
"Sydney, Australia",673
"New York City, NY",639
...,...
Mediterranean,1
"Huntington, WV",1
"Plymouth, WI",1


In [31]:
# Fill user loc com unknown
reviews['tripType'].fillna('Unkown', inplace=True)
reviews['tripType'].value_counts(dropna=False)

<ipython-input-31-bd070ecdaad1>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  reviews['tripType'].fillna('Unkown', inplace=True)


,count
tripType,
Couples,30768
Unkown,23373
Family,13001
Friends,11440
Solo,4986
Business,830


---

<font color='#0095DA'>

### **Holidays** </font> Dataset

Has discovered in the Data Understanding Notebook, this dataset has missing values within the variables ``isPaidTimeOff`` and ``countryRegionCode`` has missing values:
- ``isPaidTimeOff``: 65,623 missing values (~94% of the dataset)
- ``countryRegionCode``: 4,998 missing values (~7% of the dataset)

In [32]:
holidays.isnull().sum()

,0
countryOrRegion,0
holidayName,0
normalizeHolidayName,0
isPaidTimeOff,1389
countryRegionCode,99
date,0
translatedHolidayName,0


Since ``isPaidTimeOff`` is only available for US, GB, and India it was droped as an imputation for such high numver of missing values would lead to very skewd results.


In [33]:
holidays.drop(columns=['isPaidTimeOff'], inplace=True)
holidays.isnull().sum()

,0
countryOrRegion,0
holidayName,0
normalizeHolidayName,0
countryRegionCode,99
date,0
translatedHolidayName,0


To treat the missing values for ``countryRegionCode`` we imputed the codes that had the same values that match the ``countryOrRegion``.

In [34]:
# Step 1: Define a combined function to fill missing values for specified columns
def fill_missing_values(group, columns):
    for column in columns:  # Iterate over the columns to be processed
        if group[column].notnull().any():  # Check if there's at least one valid value
            valid_value = group.loc[group[column].notnull(), column].iloc[0]  # Get the first valid value
            group[column] = group[column].fillna(valid_value)  # Fill NaN values with the valid one
    return group

# Step 2: Specify the columns to treat and apply the function to each group of countryOrRegion
columns_to_fill = ['countryRegionCode']
holidays = holidays.groupby('countryOrRegion', group_keys=False).apply(fill_missing_values, columns=columns_to_fill)

# Step 3: Verify the changes
display(holidays[holidays['countryRegionCode'].isnull()])  # Check remaining NaN values in countryRegionCode


<ipython-input-34-2cba20e9509b>:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  holidays = holidays.groupby('countryOrRegion', group_keys=False).apply(fill_missing_values, columns=columns_to_fill)


,countryOrRegion,holidayName,normalizeHolidayName,countryRegionCode,date,translatedHolidayName
25403,England,New Year's Day,New Year's Day,NaN,2019-01-01,New Year's Day
25415,Northern Ireland,New Year's Day,New Year's Day,NaN,2019-01-01,New Year's Day
25419,Scotland,New Year's Day,New Year's Day,NaN,2019-01-01,New Year's Day
25428,Wales,New Year's Day,New Year's Day,NaN,2019-01-01,New Year's Day
25430,Scotland,New Year Holiday,New Year Holiday,NaN,2019-01-02,New Year Holiday
...,...,...,...,...,...,...
26772,Northern Ireland,Spring Bank Holiday,Spring Bank Holiday,NaN,2021-05-31,Spring Bank Holiday
26773,Scotland,Spring Bank Holiday,Spring Bank Holiday,NaN,2021-05-31,Spring Bank Holiday
26776,Wales,Spring Bank Holiday,Spring Bank Holiday,NaN,2021-05-31,Spring Bank Holiday
26821,Northern Ireland,Battle of the Boyne,Battle of the Boyne,NaN,2021-07-12,Battle of the Boyne


In [35]:
# Step 3: Verify the changes, grouped by countryOrRegion
missing_values_by_region = holidays[holidays['countryRegionCode'].isnull()] \
    .groupby('countryOrRegion').size().reset_index(name='missing_count')

display(missing_values_by_region)  # Display missing values grouped by countryOrRegion

,countryOrRegion,missing_count
0,England,22
1,Northern Ireland,29
2,Scotland,26
3,Wales,22


In [36]:
(holidays['countryRegionCode']=='GB').sum()

38

In [37]:
holidays.loc[holidays['countryOrRegion'] == 'England', ['countryRegionCode']] = ['GB']
holidays.loc[holidays['countryOrRegion'] == 'Northern Ireland', ['countryRegionCode']] = ['GB']
holidays.loc[holidays['countryOrRegion'] == 'Scotland', ['countryRegionCode']] = ['GB']
holidays.loc[holidays['countryOrRegion'] == 'Wales', ['countryRegionCode']] = ['GB']
display((holidays['countryRegionCode']=='GB').value_counts())

,count
countryRegionCode,
False,1330
True,137


In [38]:
holidays.isnull().sum()

,0
countryOrRegion,0
holidayName,0
normalizeHolidayName,0
countryRegionCode,0
date,0
translatedHolidayName,0


<a name="transformation"></a>
# 3. Data Transformation


<a name="feature"></a>
## 3.1. Feature Engeniering


In [39]:
#pd.set_option('display.max_rows', None)

In [40]:
#pd.reset_option('display.max_rows')

In [41]:
# Define a regex pattern to match rows with numbers or special characters
pattern = r'[^a-zA-Z\s,]'
# Check if rows in 'userLocation' contain numbers or special characters
contains_invalid = reviews['userLocation'].str.contains(pattern, na=False)
# Filter rows without numbers or special characters
cleaned_reviews = reviews[~contains_invalid]

print(f"Rows dropped: {contains_invalid.sum()}")
print(f"Original dataset size: {reviews.shape[0]}")
print(f"Cleaned dataset size: {cleaned_reviews.shape[0]}")

Rows dropped: 1883
Original dataset size: 84398
Cleaned dataset size: 82515


In [42]:
# Extract country in column userLocation
reviews['userCountryUntrieted'] = reviews['userLocation'].str.split(',').str[-1].str.strip()
print(reviews[['userLocation', 'userCountryUntrieted']].head(40))

                        userLocation userCountryUntrieted
0                   Indianapolis, IN                   IN
1                    Bandon, Ireland              Ireland
2                    Valencia, Spain                Spain
3                             Unkown               Unkown
4                      Madrid, Spain                Spain
5                         Towaco, NJ                   NJ
6                             Unkown               Unkown
7                       Barnsley, UK                   UK
8   Sofiyivska Borschagivka, Ukraine              Ukraine
9                             Unkown               Unkown
10                            France               France
11                      Fairport, NY                   NY
12               Zurich, Switzerland          Switzerland
13                    Athens, Greece               Greece
14                         Barcelona            Barcelona
15        Groningen, The Netherlands      The Netherlands
16            

In [43]:
# Transform first letter in every word CAPS in column 'userCountryFull'
reviews['userCountryUntrieted'] = reviews['userCountryUntrieted'].str.title()
print(reviews[['userCountryUntrieted']].head())

  userCountryUntrieted
0                   In
1              Ireland
2                Spain
3               Unkown
4                Spain


In [44]:
cities_of_the_world = pd.read_excel("/content/drive/MyDrive/DS4M_PROJECT/World_Cities.xlsx")

In [ ]:
#loop through each row of the reviews
for index, row in reviews.iterrows():
    #it gets the value in the 'userCountry' column.
    user_location = row['userCountryUntrieted']
    #It searches for a matching city in the cities_of_the_world DataFrame based on the 'userCountry'.
    match = cities_of_the_world[cities_of_the_world['city'] == user_location]

    # If a match is found, replace the userCountry with the corresponding country
    if not match.empty:
        country = match.iloc[0]['country']
        reviews.at[index, 'userCountryUntrieted'] = country

In [ ]:
print((reviews['userCountryUntrieted']).head(40))

In [ ]:
#Dictionary for siglas
state_to_country = {

    'AL': 'United States', 'AK': 'United States', 'AZ': 'United States',
    'AR': 'United States', 'CA': 'United States', 'CO': 'United States',
    'CT': 'United States', 'DE': 'United States', 'FL': 'United States',
    'GA': 'United States', 'HI': 'United States', 'ID': 'United States',
    'IL': 'United States', 'IN': 'United States', 'IA': 'United States',
    'KS': 'United States', 'KY': 'United States', 'LA': 'United States',
    'ME': 'United States', 'MD': 'United States', 'MA': 'United States',
    'MI': 'United States', 'MN': 'United States', 'MS': 'United States',
    'MO': 'United States', 'MT': 'United States', 'NE': 'United States',
    'NV': 'United States', 'NH': 'United States', 'NJ': 'United States',
    'NM': 'United States', 'NY': 'United States', 'NC': 'United States',
    'ND': 'United States', 'OH': 'United States', 'OK': 'United States',
    'OR': 'United States', 'PA': 'United States', 'RI': 'United States',
    'SC': 'United States', 'SD': 'United States', 'TN': 'United States',
    'TX': 'United States', 'UT': 'United States', 'VT': 'United States',
    'VA': 'United States', 'WA': 'United States', 'WV': 'United States',
    'WI': 'United States', 'WY': 'United States', 'DC': 'United States',

    #Brazil
    'AC': 'Brazil', 'AL': 'Brazil', 'AP': 'Brazil', 'AM': 'Brazil',
    'BA': 'Brazil', 'CE': 'Brazil', 'DF': 'Brazil', 'ES': 'Brazil',
    'GO': 'Brazil', 'MA': 'Brazil', 'MT': 'Brazil', 'MS': 'Brazil',
    'MG': 'Brazil', 'PA': 'Brazil', 'PB': 'Brazil', 'PR': 'Brazil',
    'PE': 'Brazil', 'PI': 'Brazil', 'RJ': 'Brazil', 'RN': 'Brazil',
    'RS': 'Brazil', 'RO': 'Brazil', 'RR': 'Brazil', 'SC': 'Brazil',
    'SP': 'Brazil', 'SE': 'Brazil', 'TO': 'Brazil',

    #Canadá
    'AB': 'Canada', 'BC': 'Canada', 'MB': 'Canada', 'NB': 'Canada',
    'NL': 'Canada', 'NS': 'Canada', 'NT': 'Canada', 'NU': 'Canada',
    'ON': 'Canada', 'PE': 'Canada', 'QC': 'Canada', 'SK': 'Canada',
    'YT': 'Canada'
}


In [ ]:
# Substitute using dictionary
reviews['userCountryUntrieted'] = reviews['userCountryUntrieted'].replace(state_to_country)
print(reviews['userCountryUntrieted'].value_counts())

In [ ]:
reviews['userCountryUntrieted'].describe()

In [ ]:
# Count the occurrences of each value in the 'userCountryUntrieted' column
value_counts = reviews['userCountryUntrieted'].value_counts()
# Filter values that appear exactly once and calculate their sum
count_equal_to_one = (value_counts <= 10).sum()
# Display the result
print(f"Total unique values that appear exactly once: {count_equal_to_one}")

In [ ]:
reviews['userCountryUntrieted'] = reviews['userCountryUntrieted'].replace({'The Netherlands': 'Netherlands'})
display(reviews['userCountryUntrieted'].unique())

In [ ]:
# Drop rows with low-frequency values in 'userCountryUntrieted'
value_counts = reviews['userCountryUntrieted'].value_counts()
values_to_keep = value_counts[value_counts > 10].index
reviews = reviews[reviews['userCountryUntrieted'].isin(values_to_keep)]
#updated dataset size
print(f"Updated dataset size: {reviews.shape[0]}")

In [ ]:
reviews['userCountryUntrieted'].describe()

In [ ]:
reviews['userCountryUntrieted'].value_counts()

<a name="new"></a>
## 3.3. Creating New Variables


In [ ]:
# Ensure that both 'reviewWritten' and 'reviewVisited' columns are in datetime format
reviews['reviewWritten'] = pd.to_datetime(reviews['reviewWritten'])
reviews['reviewVisited'] = pd.to_datetime(reviews['reviewVisited'])
# Create a new feature for the time difference (in days) between 'reviewWritten' and 'reviewVisited'
reviews['timeDifferenceDays'] = (reviews['reviewWritten'] - reviews['reviewVisited']).dt.days
# Display the updated dataset with the new feature
display(reviews[['reviewWritten', 'reviewVisited', 'timeDifferenceDays']].head())

In [ ]:
# Ensure both dates are in datetime format
reviews['reviewVisited'] = pd.to_datetime(reviews['reviewVisited'])
holidays['date'] = pd.to_datetime(holidays['date'])
# Create a set of unique holiday dates for quick lookup
holiday_dates = set(holidays['date'])
# Create the binary feature
reviews['visitedDuringHoliday'] = reviews['reviewVisited'].apply(lambda x: 1 if x in holiday_dates else 0)
# Check the distribution of the binary feature
print(reviews['visitedDuringHoliday'].value_counts())

For this, we'll categorize dates into seasons:

- Winter: December 21 - March 20
- Spring: March 21 - June 20
- Summer: June 21 - September 20
- Autumn: September 21 - December 20

In [ ]:
# Ensure reviewVisited is in datetime format
reviews['reviewVisited'] = pd.to_datetime(reviews['reviewVisited'])
# Function to determine season
def get_season(date):
    if pd.isna(date):  # Handle missing dates
        return None
    month = date.month
    day = date.day
    if (month == 12 and day >= 21) or (month <= 3 and day <= 20):
        return 'Winter'
    elif (month == 3 and day >= 21) or (month <= 6 and day <= 20):
        return 'Spring'
    elif (month == 6 and day >= 21) or (month <= 9 and day <= 20):
        return 'Summer'
    elif (month == 9 and day >= 21) or (month <= 12 and day <= 20):
        return 'Autumn'

# Create the season feature
reviews['seasonVisited'] = reviews['reviewVisited'].apply(get_season)
print(reviews['seasonVisited'].value_counts())

In [ ]:
# Ensure the 'date' column is in datetime format
holidays['date'] = pd.to_datetime(holidays['date'])
# Create a new feature for the month and day (as a string in 'MM-DD' format)
holidays['monthDay'] = holidays['date'].dt.strftime('%m-%d')
# Display the updated dataset with the new feature
display(holidays[['date', 'monthDay']].sample())
display(holidays[['monthDay']].value_counts())

In [ ]:
# Exportar o DataFrame 'reviews' para um arquivo Excel
reviews.to_excel('reviews_cleaned.xlsx', index=False)

In [ ]:
reviews_cleaned = pd.read_excel('reviews_cleaned.xlsx')
reviews_cleaned.info()

In [ ]:
attractions.to_excel('attractions_cleaned.xlsx', index = False)

In [ ]:
attractions_cleaned = pd.read_excel('attractions_cleaned.xlsx')
attractions_cleaned.info()

In [ ]:
holidays.to_excel('holidays_cleaned.xlsx', index = False)

In [ ]:
holidays_cleaned = pd.read_excel('holidays_cleaned.xlsx')
holidays_cleaned.info()

<a name="integration"></a>
# 4. Data Integration


<a name="merge"></a>
## 4.1. Merging Datasets

In [ ]:
reviews_cleaned['localID'] = reviews_cleaned['localID'].str.strip().str.upper()
attractions_cleaned['ID'] = attractions_cleaned['ID'].str.strip().str.upper()
attractions_cleaned['Country'] = attractions_cleaned['Country'].str.strip().str.upper()
holidays_cleaned['countryOrRegion'] = holidays_cleaned['countryOrRegion'].str.strip().str.upper()

In [ ]:
reviews_attractions = pd.merge(reviews_cleaned, attractions_cleaned, left_on='localID', right_on='ID', how='left')

In [ ]:
columns_to_remove = ['ID']
reviews_attractions = reviews_attractions.drop(columns=columns_to_remove)

In [ ]:
reviews_attractions.info()

In [ ]:
# Exportar o DataFrame 'reviews' para um arquivo Excel
reviews_attractions.to_excel('reviews_attractions.xlsx', index=False)

<a name="scaling"></a>
# 5. Data Scalling/Normalization


#Feature Selection
#Correlation Analysis

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Selecionar apenas colunas numéricas do dataset
numeric_cols = reviews_attractions.select_dtypes(include=[np.number])

# Calcular a correlação de Spearman
cor_spearman = numeric_cols.corr(method='spearman')

# Plotar o triângulo inferior do mapa de calor de correlação
plt.figure(figsize=(26, 18))
mask = np.triu(np.ones_like(cor_spearman, dtype=bool))  # Máscara para triângulo superior
heatmap = sns.heatmap(cor_spearman, mask=mask, vmin=-1, vmax=1, annot=True, cmap='vlag', fmt=".2f")
heatmap.set_title("Spearman's Correlation Heatmap", fontdict={'fontsize': 18}, pad=16)
plt.show()

# Função para filtrar correlações acima de um limiar (threshold)
def corrFilter(df, threshold):
    correlation_matrix = df.corr(method='spearman')
    # Filtrar correlações acima/abaixo do limiar (excluindo 1.0 e valores nulos)
    filtered_matrix = correlation_matrix[
        ((correlation_matrix >= threshold) | (correlation_matrix <= -threshold))
        & (correlation_matrix != 1.000)
        & (correlation_matrix.isnull() == False)
    ]
    # Desempilhar e organizar os pares de correlação
    corr_pairs = (
        filtered_matrix.unstack()
        .sort_values(key=abs, ascending=False)
        .drop_duplicates()
        .to_frame(name="Spearman's Correlation")
    )
    return corr_pairs.iloc[:-1, :]  # Remover a última linha redundante

# Aplicar a função ao dataset numérico
filtered_correlations = corrFilter(numeric_cols, threshold=0.7)

# Exibir as correlações filtradas
print(filtered_correlations)


In [ ]:
# Gerar a matriz de correlação (Spearman)
cor_spearman = numeric_cols.corr(method='spearman')

# Aplicar filtro para correlações altas (exemplo: |correlation| > 0.85)
high_correlation = cor_spearman[(cor_spearman >= 0.85) | (cor_spearman <= -0.85)]

# Exibir pares com correlação alta
print("Pares com correlação alta:")
print(high_correlation.unstack().dropna().sort_values(ascending=False))


In [ ]:
# Lista de colunas a serem removidas
columns_to_drop = ['extractionDate', 'sitesOnRanking', 'userLocation',
                   'reviewWritten', 'reviewLanguage', 'reviewFullText']

# Remover as colunas do dataset
reviews_attractions = reviews_attractions.drop(columns=columns_to_drop)
display(reviews_attractions.head())

<a name="final-review"></a>
# 6. Final Dataset Review and Export

<a name="summary-data"></a>
## 6.1. Summary of Preprocessed Data


<a name="saving-data"></a>
## 6.2. Saving the Cleaned Datasets

In [ ]:
# Exportar a folha Reviews para um novo arquivo Excel
reviews_attractions.to_excel('Reviews_Attractions_Final.xlsx', index=False)

In [ ]:
files.download('Reviews_Attractions_Final.xlsx')

In [ ]:
# Mount Google Drive
drive.mount('/content/drive')

# Save to a folder in your Drive
reviews_attractions.to_excel('/content/drive/My Drive/DS4M_PROJECT/Reviews_Exported_v2.xlsx', index=False)